In [72]:
from stox.imports import *
from stox.api import IB_Extractor
from stox import misc, yahoo
from omnibelt import tqdmd_notebook as tqdmd
from collections import Counter
# from stox.alpha_vantage import AlphaVantageStocks

yahoo_root = misc.yahoo_root()
yahoo_root = yahoo_root.parent.parent / 'testdata' / 'yahoo_data'
print(yahoo_root)

C:\Users\anwan\OneDrive\Khan\projects\stox\testdata\yahoo_data


In [191]:
root = misc.assets_root()
path = root / 'symbols.yml'
data = load_export(path)
existing = Counter(row['yahoo'] for row in data)
len(data), len(existing), existing.most_common(2)

(176, 176, [('AAPL', 1), ('MC.PA', 1)])

In [182]:
matches = {}

In [186]:
raw = '''BLG.BR	BLG.BR
AGB.BR	AGB.BR
TNO.BR	TNO.BR
UCB.BR	UCB SA
AVH.BR	AVH.BR
GBL.BR	GBL.BR
SOL.BR	SOL.BR
INT.BR	INT.BR
GLS.BR	GLS.BR
KBC.BR	KBC Group NV
UMC.BR	UMC.BR
DEL.BR	DEL.BR
ENGI.BR	ENGI.BR
ONTEX.BR	Ontex Group NV
COFB.BR	Cofinimmo SA
ES6.BR	ES6.BR
BEK.BR	BEK.BR
BPO.BR	BPO.BR
COL.BR	COL.BR
INGA.BR	ING Groep NV'''.split('\n')
companies = [row.split('\t') for row in raw]
companies = [(tk, name.lower().replace('.br','').replace('.', '').replace(' spa', '').replace(' nv', '').replace(' sa', '').replace(' plc', '').replace(' aktiengesellschaft', '').replace(' ag', '').replace(' se', '')) for tk, name in companies]
companies

[('BLG.BR', 'blg'),
 ('AGB.BR', 'agb'),
 ('TNO.BR', 'tno'),
 ('UCB.BR', 'ucb'),
 ('AVH.BR', 'avh'),
 ('GBL.BR', 'gbl'),
 ('SOL.BR', 'sol'),
 ('INT.BR', 'int'),
 ('GLS.BR', 'gls'),
 ('KBC.BR', 'kbc group'),
 ('UMC.BR', 'umc'),
 ('DEL.BR', 'del'),
 ('ENGI.BR', 'engi'),
 ('ONTEX.BR', 'ontex group'),
 ('COFB.BR', 'cofinimmo'),
 ('ES6.BR', 'es6'),
 ('BEK.BR', 'bek'),
 ('BPO.BR', 'bpo'),
 ('COL.BR', 'col'),
 ('INGA.BR', 'ing groep')]

In [187]:
# companies = 
todo = [c[1] for c in companies]

In [131]:
ibe = IB_Extractor(True)

ERROR:ib_insync.client: Peer closed connection. clientId 1 already in use?
ERROR:ib_insync.client: API connection failed: TimeoutError()


TimeoutError: 

In [188]:
# cts = ibe.search('Dr. Ing. h.c. F. Porsche')
def get_matches(query):
    cts = ibe.search(query)
    matches = list(misc.best_matches([c for c in cts if c.secType == 'STK' and c.currency == 'EUR'], [query],
                                     key1=lambda c: c.description))
    return matches

In [189]:
for query in tqdmd(todo, key=True):
    if not ibe.ib.isConnected():
        ibe = IB_Extractor(True)
    if query not in matches:
        try:
            matches[query] = get_matches(query)
        except ConnectionError:
            matches[query] = []

  0%|          | 0/20 [00:00<?, ?it/s]

In [190]:
tbl = [(query, tk, [m[1].symbol for m in matches.get(query, [])]) for tk, query in companies]
print(tabulate(tbl))

-----------  --------  ----------------------------------------------------------
blg          BLG.BR    []
agb          AGB.BR    ['D6D', 'GGV9']
tno          TNO.BR    ['TNO', 'TNOW']
ucb          UCB.BR    ['UCB', 'UCBN']
avh          AVH.BR    ['AVH']
gbl          GBL.BR    ['GBLB', 'ISPA', 'PR1W', 'FGEQ', 'HAGG', 'GISE']
sol          SOL.BR    ['SOL', 'SOLB', 'SOLV', 'SOL1', '13X']
int          INT.BR    ['INT', 'INTO', 'INTRK', 'RMS', 'ISP']
gls          GLS.BR    []
kbc group    KBC.BR    ['KBC']
umc          UMC.BR    ['UMCB', 'UM3']
del          DEL.BR    ['DELG', 'DEL2', 'DLS', 'AD', 'DHZ', 'DHER', '98D']
engi         ENGI.BR   ['ENGI', 'MTX']
ontex group  ONTEX.BR  ['ONTEX']
cofinimmo    COFB.BR   ['COFB', 'COFB.DRTS', 'COFB.RTS3', 'COFB.RTS', 'COFB.DIV']
es6          ES6.BR    []
bek          BEK.BR    ['BEKB', 'TERB.DRTS', 'TERB.DIVR', 'TERB.RTS']
bpo          BPO.BR    ['BPOST']
col          COL.BR    ['COL', 'COLR', 'CCEP', 'EEE1']
ing groep    INGA.BR   ['INGA', 'INNA']

In [192]:
bad = []
for tk, query in companies:
    if tk in existing:
        continue
    ms = matches.get(query, [])
    if len(ms) == 0:
        bad.append((tk, query))
        print(tk, query)
    else:
        existing[tk] += 1
        data.extend({'ibkr': m[1].symbol, 'yahoo': tk} for m in ms)
for tk, q in bad:
    data.append({'ibkr': tk.split('.')[0], 'yahoo': tk})
    existing[tk] += 1
len(data)

BLG.BR blg
GLS.BR gls
ES6.BR es6


231

In [193]:
export(data, path);

In [194]:
print(tabulate(bad))

------  ---
BLG.BR  blg
GLS.BR  gls
ES6.BR  es6
------  ---


In [201]:
out = ibe.search('UMI')
options = [c for c in out if c.secType == 'STK']
print(tabulate([(c.symbol, c.currency, c.description) for c in reversed(options)]))

-----  ---  ----------------------------
UMIN   MXN  UMICORE
UMI.B  CAD  CI WISDOMTREE US MID DIV-B
UMICY  USD  UMICORE - UNSPON ADR
UMI    CAD  CI WISDOMTREE US MID DIV
UMI    USD  USCF MIDSTREAM ENERGY INCOME
UMI    EUR  UMICORE
-----  ---  ----------------------------


In [102]:
ibe.ib.disconnect()